<a href="https://colab.research.google.com/github/HambaliMarcel/GOLDMINE-WebProject-Django-Amanco2021/blob/main/competency_gap_resolution_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai==0.28
!pip install fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


In [3]:
#FINAL Model PoC LNA 12 May 2024 - Need Approval
import pandas as pd
import openai
from google.colab import files

In [ ]:
api_key = " "

In [ ]:
lna_master = pd.read_csv('lna_master.csv')
lna_score = pd.read_csv('lna_score.csv')

In [ ]:
merged_data = pd.merge(lna_master, lna_score, on='competency_id', how='inner')

In [ ]:

openai.api_key = api_key


In [ ]:

def generate_learning_solutions(row):
    if row['status']:
        required_level_map = {1: 'Entry', 2: 'Basic', 3: 'Intermediate', 4: 'Advanced', 5: 'Expert'}
        required_level_description = required_level_map.get(row['required_level'])
        current_level_map = {1: 'Entry', 2: 'Basic', 3: 'Intermediate', 4: 'Advanced', 5: 'Expert'}
        current_level_description = required_level_map.get(row['current_level'])
        if row['kelas_jabatan'] > 15:
            prompt = f"{row['employee_id']} requires enhancement in the competency {row['competency_id']}, with an actual level of {current_level_description} compared to the required mastery level of {required_level_description}.  I need brief recommendations (only 2 sentences) for a targeted learning solution approach tailored to {row['competency_name']} at a {required_level_description} proficiency level, Consider that {row['rumpun_jabatan']} as a job family. The solution should prioritize non-technical and efficient learning methods and Consider difficulty of the solution to fit for {required_level_description} proficiency level."
        else:
            prompt = f"{row['employee_id']} requires enhancement in the competency {row['competency_id']}, with an actual level of {current_level_description} compared to the required mastery level of {required_level_description}.  I need brief recommendations (only 2 sentences) recommendations for a targeted learning solution approach tailored to {row['competency_name']} at a {required_level_description} proficiency level, Consider that {row['rumpun_jabatan']} as a job family. Consider difficulty of the solution to fit for {required_level_description} proficiency level."
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.5,
            max_tokens=150
        )
        return response.choices[0].text.strip()
    else:
        return ''


In [ ]:

def generate_learning_topics(row):
    if row['status']:
        required_level_map = {1: 'Entry', 2: 'Basic', 3: 'Intermediate', 4: 'Advanced', 5: 'Expert'}
        required_level_description = required_level_map.get(row['required_level'])
        prompt = f"What are the relevant key outlines or latest essential knowledge or paradigm that I will acquire for the competency {row['competency_name']} at the {required_level_description} proficiency level? List in 1 Title learning and 5 point of brief outline. Make the outline simple (just one line)"
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.5,
            max_tokens=100
        )
        topics = [topic['text'].strip() for topic in response.choices[:2]]
        return '; '.join(topics)
    else:
        return ''


In [ ]:

merged_data['recommendation'] = merged_data.apply(generate_learning_solutions, axis=1)
merged_data['topic_1'] = merged_data.apply(generate_learning_topics, axis=1)


merged_data = merged_data[merged_data['recommendation'] != '']


In [ ]:

output_file = 'learning_recommendations.xlsx'
selected_columns = ['lna_master_id', 'employee_id', 'group_id', 'position_id', 'competency_id', 'name', 'kelas_jabatan', 'rumpun_jabatan', 'vitality_score', 'priority_comp', 'current_level', 'required_level', 'status', 'aggregate_competency', 'lna_batch_id_x', 'createdAt_x', 'updatedAt_x', 'lna_score_id', 'competency_name', 'recommendation', 'topic_1']
merged_data[selected_columns].to_excel(output_file, index=False)

files.download(output_file)